### 전처리

In [1]:
import pymongo as mg
import pandas as pd

In [2]:
client = mg.MongoClient(host='mongodb://localhost:27017')

In [3]:
database = client['db_NHIS']
collection = database['NSC2_BND_M20']

In [4]:
cursor = collection.find({})
list_BND_M20 = list(cursor)

In [5]:
df_BND_M20 = pd.DataFrame(list_BND_M20)
df_BND_M20[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,EDC_ADD_RT,ED_RC_TOT_AMT,SICK_SYM1
0,64ed6a2652642d1f5783b3b2,294364,1921LE,200707.0,T08-T14,W00-W19,0.15,10590,L028
1,64ed6a2652642d1f5783b3b3,294364,1921LE,200707.0,T08-T14,W00-W19,0.15,16520,J030


In [6]:
df_BND_M20['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [7]:
len('F_'), len('J209'), 'J209'[:3], str('T18')[:3]

(2, 4, 'J20', 'T18')

In [8]:
def convertSICK_SYM(sick_sym) :
    if len(sick_sym) < 3:
        return pd.NA  #  np.Nan None
    else :
        return sick_sym[:3]

In [9]:
convertSICK_SYM('J209'), convertSICK_SYM('F_')

('J20', <NA>)

In [10]:
df_BND_M20['SICK_SYM1_3'] = df_BND_M20['SICK_SYM1'].apply(convertSICK_SYM)

In [11]:
df_BND_M20['SICK_SYM1_3'].isnull().sum()

11354

In [12]:
df_BND_M20['SICK_SYM1_3'].value_counts()

J20    8917
I10    8427
J03    4187
M54    3904
J06    3460
       ... 
G97       1
K01       1
S28       1
I72       1
P28       1
Name: SICK_SYM1_3, Length: 747, dtype: int64

In [13]:
df_BND_M20_dropna = df_BND_M20.dropna()
df_BND_M20_dropna.isnull().sum()

_id              0
RN_INDI          0
BTH_YYYY         0
DTH_YYYYMM       0
COD1             0
COD2             0
EDC_ADD_RT       0
ED_RC_TOT_AMT    0
SICK_SYM1        0
SICK_SYM1_3      0
dtype: int64

In [14]:
df_BND_M20_dropna[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,EDC_ADD_RT,ED_RC_TOT_AMT,SICK_SYM1,SICK_SYM1_3
0,64ed6a2652642d1f5783b3b2,294364,1921LE,200707.0,T08-T14,W00-W19,0.15,10590,L028,L02
1,64ed6a2652642d1f5783b3b3,294364,1921LE,200707.0,T08-T14,W00-W19,0.15,16520,J030,J03


In [15]:
target_list = df_BND_M20_dropna['SICK_SYM1_3'].value_counts().index.to_list()

In [16]:
target_list.index('L02'), target_list.index('J03'), 

(88, 4)

In [17]:
def applyIndexNumber(sick_sym_3):
    indexNumber = target_list.index(sick_sym_3)
    return indexNumber

In [18]:
df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)
df_BND_M20_dropna[:2]

C:\Users\05-16\AppData\Local\Temp\ipykernel_16192\836018491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)


,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,EDC_ADD_RT,ED_RC_TOT_AMT,SICK_SYM1,SICK_SYM1_3,SICK_SYM1_3_target
0,64ed6a2652642d1f5783b3b2,294364,1921LE,200707.0,T08-T14,W00-W19,0.15,10590,L028,L02,88
1,64ed6a2652642d1f5783b3b3,294364,1921LE,200707.0,T08-T14,W00-W19,0.15,16520,J030,J03,4


### 정형화

In [19]:
df_BND_M20_dropna.columns

Index(['_id', 'RN_INDI', 'BTH_YYYY', 'DTH_YYYYMM', 'COD1', 'COD2',
       'EDC_ADD_RT', 'ED_RC_TOT_AMT', 'SICK_SYM1', 'SICK_SYM1_3',
       'SICK_SYM1_3_target'],
      dtype='object')

In [42]:
target_temp = pd.DataFrame(df_BND_M20_dropna['SICK_SYM1_3_target'])
features = df_BND_M20_dropna[['EDC_ADD_RT', 'ED_RC_TOT_AMT',]]
target_temp.shape, features.shape

((897, 1), (897, 2))

In [43]:
from sklearn.preprocessing import OneHotEncoder
encoder_target = OneHotEncoder()
target = encoder_target.fit_transform(target_temp).toarray()
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((672, 2), (672, 151), (225, 2), (225, 151))

#### 모델학습 

In [45]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [46]:
model = DecisionTreeClassifier()

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
hyper_params = {'min_samples_leaf' : range(2,5)
               , 'max_depth' : range(2,5)  # [2,3,4,5]
               , 'min_samples_split' : range(2,5)}

##### 평가 score Default : 분류-Accuracy, 예측-R squre

In [49]:
from sklearn.metrics import f1_score, make_scorer

In [50]:
scoring = make_scorer(f1_score)

In [51]:
grid_search = GridSearchCV(model, param_grid=hyper_params, cv=2
                           , verbose=1, scoring=scoring)

In [29]:
# grid_search.fit(features_train, target_train)
grid_search.fit(features, target)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


C:\Users\05-16\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\05-16\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\05-16\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\05-16\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\05-16\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\05-16\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1123, in f1_sc

GridSearchCV(cv=2, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(2, 5),
                         'min_samples_leaf': range(2, 5),
                         'min_samples_split': range(2, 5)},
             scoring=make_scorer(f1_score), verbose=1)

In [30]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2)

In [31]:
grid_search.best_score_, grid_search.best_params_

(nan, {'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2})

In [32]:
best_model = grid_search.best_estimator_
best_model

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2)

In [33]:
target_test_predict = best_model.predict(features_test)
target_test_predict

array([0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 5, 0, 0, 0,
       7, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0,
       0, 0, 7, 7, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0,
       0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0,
       0, 9, 7, 0, 7, 9, 9, 0, 0, 0, 0, 0, 0, 0, 9, 0, 5, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 5, 0, 7, 0, 0, 0, 7,
       0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 5, 9, 9, 0, 7, 0,
       0, 0, 0, 0, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0,
       0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       7, 9, 0, 0, 0], dtype=int64)

In [34]:
from sklearn.metrics import classification_report

In [35]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.12      1.00      0.21        22
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00        11
           5       0.64      0.78      0.70         9
           6       0.00      0.00      0.00         6
           7       0.32      1.00      0.48         6
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00         4
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         5
          16       0.00      0.00      0.00         3
          17       0.00      0.00      0.00         5
          18       0.00    

C:\Users\05-16\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\05-16\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\05-16\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
